In [1]:
import os
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import factorial
from scipy import optimize
from matplotlib.pyplot import cm
from lmfit import Model
from matplotlib.gridspec import GridSpec
from scipy.stats import poisson as pois
from scipy.stats import norm

## Load Dataset

In [2]:
# load dataset including metadata info
def load_dataset(run, path='db/'):
    file = f'{path}AnalyzedDataFFT_Run_{run}_sliced.xlsx'
    meta = pd.read_excel(file,sheet_name=0,header=None)
    freq = pd.read_excel(file,sheet_name=1)             # frequecies
    fft  = pd.read_excel(file,sheet_name=2)             # power
    
    data = pd.DataFrame({'freq':freq[1]})
    
    col = 0
    for col_name in fft.columns: # load all the subruns
        if col > 0:
            data[f'fft{col-1}'] = fft[col_name]
        col += 1
    
    #cavity frequency and number of files in each slice
    center = meta[1][3]
    length = meta[1][8]
    
    print(f'Dataset loaded: {file}\nCavity frequency: {center} Hz\nfft subrun:{len(data.columns)-1}')
    
    return data,center,length

## Background and Signal Fits

### Prepare Data

In [ ]:
def prep_data(alldata,center,subrun=0,length=500,bin_width=651,nbins=100):
    
    N = length*2731 #N=1365500 if length=500
    
    # select window of 2*nbins bins around center
    # default is to select 200 bins of 651 Hz
    mask = (alldata['freq']>center-bin_width*nbins) & (alldata['freq']<center+bin_width*nbins)
    cavdata = alldata[mask].reset_index()
    
    # scale data to yottowat
    minW = np.min(cavdata[f'fft{subrun}'].copy()) # minimum power in the cavity
    
    # In general, the average measured power should be known and equal to the noise temperature of the system.
    # So we can rescale the data so that the power at the cavity frequency sia T_noise k_b B (W)
    ref = minW**(-1) * 3.5*1.38e-23*651/1e-24 #It is possibile to add an extra contribute to make them integers
    cavdata[f'fft{subrun}'] = ref * cavdata[f'fft{subrun}']   #y' 
    
    # set weights
    #weights = cavdata[f'fft{subrun}']/np.sqrt(N)      # -> y'/sqrt(N)
    weights = np.sqrt(ref)*np.sqrt(cavdata[f'fft{subrun}'])/np.sqrt(N)  #-> sqrt(sigma'/N)=ref*sqrt(y/N)
    
    #This function returns freq,fft and weights
    freq = cavdata['freq']
    fft=cavdata[f'fft{subrun}']
    
    return(freq, fft, weights,ref)

### Fit Functions

In [ ]:
def gaussian(x,x0,s,mu):
    return mu * np.exp(-.5*((x-x0)/s)**2)
    
def maxwell(x,x0,s,mu):
    return mu * x**2/s**3 * np.exp(-.5*((x-x0)/s)**2)
    
def bkg(x,a,b,c,d,e,f):                                                   
    return e**2*abs(x-a+1j*b)**2/abs(x-c+1j*d)**2+f*(x-c)
    
def signal_gauss(x,a,b,c,d,e,f,x0,s,mu):
    return bkg(x,a,b,c,d,e,f) + gaussian(x,x0,s,mu)
    
def signal_maxwell(x,a,b,c,d,e,f,x0,s,mu):
    return bkg(x,a,b,c,d,e,f) + maxwell(x,x0,s,mu)

### Fit Background

In [ ]:
def fit_bkg(freq, fft, weights, center, ref):
    # set fit model
    bkg_model = Model(bkg)
    ps = bkg_model.make_params(a={'value':center, 'min':center*0.999, 'max':center*1.01},
                               b=2e4,
                               c={'value':center, 'min':center*0.999, 'max':center*1.01},
                               d=2.2e4,
                               e=1e-2*np.sqrt(ref),
                               f=1e-12*ref)
    # execute fit
    result = bkg_model.fit(fft,x=freq,params=ps,weights=1/weights)
    
    return(result)

### Fit Signal

In [ ]:
def fit_sig(freq, fft, weights, x_0, res_bkg, signal, mu_init = 1e-8, mu_vary = True):
    
    # take result of preliminary background fit to fix starting parameters
    p = res_bkg.best_values
    
    # set fit model
    sig_model = Model(signal)
    ps = sig_model.make_params(a ={'value':p['a'], 'vary':False},
                               b ={'value':p['b'], 'vary':False},
                               c ={'value':p['c'], 'vary':False},
                               d ={'value':p['d'], 'vary':False},
                               e ={'value':p['e'], 'vary':False},
                               f ={'value':p['f'], 'vary':False},
                               mu={'value':mu_init, 'min':0, 'vary':mu_vary},
                               x0={'value':x_0, 'vary':False},
                               s ={'value':16*651, 'vary':False}) # fixed value to 16 bins

    result = sig_model.fit(fft,x=freq,params=ps,weights=1/weights)
    
    return(result)

### Plot Fit Results

In [ ]:
def plot_fit(freq, fft, weights, fit_result):
    # prepare canvas
    fig = plt.figure(figsize=(15,10))
    gs = GridSpec(2, 2)
    ax  = fig.add_subplot(gs[0,:])
    ax1 = fig.add_subplot(gs[1,0])
    ax2 = fig.add_subplot(gs[1,1])
    
    # plot data and best fit
    ax.plot(freq,fft,'o',label='data')
    ax.plot(freq,fit_result.best_fit,color='red',label='fit')
    
    fmin,fmax = min(freq),max(freq)
    ax.set_xlim([fmin,fmax])
    ax.legend()
    
    #residuals w.r.t. freq
    fit_result.plot_residuals(ax=ax1)
    
    
    # plot histogrm of residuals (with the fit)
    #rangeMax=int(np.max(res_bkg.residual))+1
    
    ax2.hist(fit_result.residual,bins=30,density=True)#range=(-rangeMax,rangeMax))
    
    
    #fit_res = norm.fit(res_bkg.residual, loc=0, scale=1)
    #ax2.plot(np.linspace(-rangeMax,rangeMax,100),norm.pdf(np.linspace(-rangeMax,rangeMax,100),fit_res[0],fit_res[1]),color="red")
    #ax2.axvline(fit_res[0], color='black', linestyle='dashed', linewidth=1)
    
    #summary_text = "mean: {}\n std: {}".format(np.round(fit_res[0],3),np.round(fit_res[1],3))
    #ax2.text(0.9, 0.9, summary_text, transform=fig.gca().transAxes, ha='right', va='top')


    # plot of residuals vs. freq and error band
    fig3,ax3=plt.subplots(1,1,figsize=(18,5))
    
    ax3.scatter(freq,fit_result.residual*weights,label="residuals")
    ax3.plot(freq, weights,label="+$\sigma$")
    ax3.plot(freq,-weights,label="-$\sigma$")
    ax3.set_xlim([fmin,fmax])
    ax3.set_title("Residuals vs freq")
    ax3.legend()
    
    plt.show()
    return(fit_result)

## Significance Test

### Generate Toy Datasets

In [ ]:
def gen_toydataset(values, n):
    toy_dataset = pois.rvs(mu=values, size=(n,len(values)))
    return(toy_dataset)

### Likelihood Ratio

In [ ]:
# compute likelihood ratios of two hypotheses
def lh_ratio(fft, model_null, model_alt):
    # compute log likelihoods
    LogLike_null = sum(pois.logpmf(fft.astype(int), model_null.best_fit.astype(int)))
    LogLike_alt  = sum(pois.logpmf(fft.astype(int), model_alt.best_fit.astype(int)))
    
    # significance of the test
    q = -2 * (LogLike_null - LogLike_alt)
    
    return(q)

### Plot Significance Distribution

In [ ]:
def plot_significance(q0, q0_obs):
    # prepare canvas
    fig = plt.figure(figsize=(15,10))
    
    # plot q0 distribution
    N = len(q0)
    binning = int(np.sqrt(N))
    n, bins, _ = plt.hist(q0, bins=binning, density = True, 
                          facecolor='lightblue', edgecolor='black', label='Toy Experiments')
    plt.vlines(q0_obs, 0, max(n), colors='blue', linestyles='--', label='Observed Data')
    
    plt.legend(loc='upper left')
    plt.xlabel('q0')
    plt.ylabel('PDf')
    
    plt.show()

### Significance

In [ ]:
def significance(freq, fft, weights, x_0, res_bkg, signal, n=1000, draw = False):
    res_sig = fit_sig(freq, fft, weights, x_0, res_bkg, signal)
    
    # compute likelihood ratio of observed data
    q0_obs = lh_ratio(fft, res_bkg, res_sig)
    
    # generate toy dataset and compute likelihood ratio for all of them
    toy_fft = gen_toydataset(res_bkg.best_fit, n)
    toy_weights = toy_fft/np.sqrt(1365500)
    
    q0 = np.zeros(n)
    for i in range(n):
        toy_bkg = fit_bkg(freq, toy_fft[i], toy_weights[i], center, ref)
        toy_sig = fit_sig(freq, toy_fft[i], toy_weights[i], x_0, toy_bkg, signal)
        q0[i] = lh_ratio(toy_fft[i], toy_bkg, toy_sig)
    
    # plot significance distribution
    if(draw):
        plot_significance(q0, q0_obs)
        
    # compute p-value
    p = sum(q0 >= q0_obs)/n
    # compute significance
    z = norm.ppf(1-p)
        
    return(z)

### Test

In [ ]:
def sig_test(run, signal, path='db/', n=1000):
    data,center = load_dataset(run, path)
    freq, fft, weights, ref = prep_data(data,center)
    res_bkg = fit_bkg(freq, fft, weights, center, ref)
    
    z = np.zeros(len(freq))
    for i in range(len(freq)):
        z[i] = significance(freq, fft, weights, freq.values[i], res_bkg, signal, n)
    
    return(z)

# FIT run 395

Runs:
- 389
- 392
- 394
- 395
- 397
- 399
- 407
- 409
- 411
- 413
- 415

In [ ]:
data,center,length = load_dataset(395)
data

## Background

In [ ]:
freq, fft, weights,ref= prep_data(data,center)

In [ ]:
res_bkg = fit_bkg(freq, fft, weights, center, ref)
plot_fit(freq, fft, weights, res_bkg)

## Signal + background

In [ ]:
res_sig = fit_sig(freq, fft, weights, center, res_bkg, signal_maxwell)
plot_fit(freq, fft, weights, res_sig)

# SIGNIFICANCE TEST - run 395

In [ ]:
q0_obs = lh_ratio(fft, res_bkg, res_sig)
q0_obs

In [ ]:
z = significance(freq, fft, weights, center, res_bkg, signal_maxwell, 100, True)
z